#### A

In [1]:
import jajucha2
import numpy as np

#### B

In [ ]:
import jajucha2
import time
import cv2

# 초점 거리(focal length)와 실제 물체의 높이를 사용하여 거리 계산 (예: 병 높이)
KNOWN_HEIGHT = 30  # 물체의 실제 높이(cm)
FOCAL_LENGTH = 500  # 카메라 초점 거리 (실험적으로 설정)

def calculate_distance(bbox_height, known_height=KNOWN_HEIGHT, focal_length=FOCAL_LENGTH):
    """
    물체의 바운딩 박스 높이를 사용하여 거리를 계산합니다.
    :param bbox_height: 바운딩 박스의 픽셀 높이
    :param known_height: 물체의 실제 높이 (cm)
    :param focal_length: 카메라의 초점 거리
    :return: 추정된 거리 (cm)
    """
    return (known_height * focal_length) / bbox_height

while True:
    frame = jajucha2.camera.get_image()

    # YOLOv8 추론 실행
    results = jajucha2.ai.get_model_output(frame, 'bottle')

    # 탐지 결과를 처리 및 시각화
    for result in results:
        for box in result.boxes:
            # 바운딩 박스 좌표 추출
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 좌표
            confidence = box.conf[0]  # 신뢰도
            class_id = int(box.cls[0])  # 클래스 ID

            # 바운딩 박스 높이 계산
            bbox_height = y2 - y1

            # 거리 계산
            distance = calculate_distance(bbox_height)

            # 바운딩 박스와 거리 시각화
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"Dist: {distance:.1f} cm"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 프레임 표시
    jajucha2.camera.show_image(frame)
